In [ ]:
import pandas as pd


start_index = 0
end_index = 50
criteria = 'gangmu_desk'
model = 'gpt-4.1-2025-04-14'
template = 'gangmu_desk'
file_path = './buffer/to_do_more.xlsx'
output_path = './buffer/gangmu_more.csv'

to_analyze = True
to_translate_comment = True
to_check = True

In [ ]:

# df = pd.read_csv(file_path, sep=None, engine='python')
# print(df.columns)
# print(df.shape)

df = pd.read_excel(file_path, sheet_name='钢木桌测试')

df = df[df['好差评结果'].isna()]
df = df.sample(n=50, replace=True)
print(df.columns)
print(df.shape)

if True:
    # df = df[['评论内容', '评分', '评论时间']]
    df['英文分析'] = ''
    df['好差评结果'] = ''
    df['审查结果'] = ''

In [ ]:
import re
import yaml
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


def remove_duplicate_values(input_dict):
    seen_values = set()  # 用于存储已经出现过的值
    output_dict = {}  # 用于存储最终的结果

    for key, value in input_dict.items():
        if value not in seen_values:
            seen_values.add(value)
            output_dict[key] = value

    return output_dict


with open('./openai_keys.yaml', "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)
my_key = data['openai']['key']

comment_analyzor = OpenAICommentAnalysisAgent(openai_key=my_key, criteria=criteria, model=model, template=template)


for i in tqdm(range(start_index, min(len(df), end_index))):
    comment = df.iloc[i]['评论内容']
    if not isinstance(comment, str):
        continue

    # 分析评论
    if to_analyze:
        analysis = comment_analyzor.comment_analyze(comment)
        df.loc[i, '英文分析'] = analysis
        # print(f"分析结果: {analysis}")

        # 提取结果
        extracted_output = extract_output(analysis)
        extracted_output = ''.join(extracted_output)
        extracted_output = eval(extracted_output)
        extracted_output = remove_duplicate_values(extracted_output)
        df.loc[i, '好差评结果'] = str(extracted_output)
        # print(f"提取结果: {extracted_output}")

    # 翻译
    if to_translate_comment:
        translated_comment = comment_analyzor.translate(comment)
        df.loc[i, '中文评论'] = translated_comment
        # print(f"原: {comment}\n翻译评论: {translated_comment}")

    # 反审查
    if to_check:
        extracted_output = df.loc[i, '好差评结果']
        check_result = comment_analyzor.inspect(comment, str(extracted_output))
        df.loc[i, '审查结果'] = check_result
df.to_csv(output_path, sep='\t', index=False)

In [ ]:
import pandas as pd


def amz_xlsx_enter(file_path, time_thre='2023-01-01'):
    xls = pd.ExcelFile(file_path)
    all_sheet_names = xls.sheet_names
    print(all_sheet_names)
    # print("文件中存在的sheet有：", all_sheet_names)

    df = pd.read_excel(file_path, sheet_name=all_sheet_names[0])
    print(df.shape)
    print(df.columns)

    df = df.rename(columns={'内容': '评论内容', '星级': '评分'})
    df['评论时间'] = pd.to_datetime(df['评论时间'], format='%Y-%m-%d')
    filtered_df = df[df["评论时间"] > pd.to_datetime(time_thre)]
    filtered_df = filtered_df.reset_index(drop=True)
    print('Before Time filtering:', sep=' ')
    print(df.shape)
    print('After Time filtering:', sep=' ') 
    print(filtered_df.shape)
    
    return filtered_df

file_path = './buffer/B0CLNJM9HP-US-Reviews-20250515.xlsx'
df = amz_xlsx_enter(file_path, time_thre='2023-01-01')
print(df.columns)
print(df['ASIN'].value_counts())
